In [1]:
from IPython.core.debugger import set_trace

#%load_ext nb_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

plt.style.use(style="seaborn")
%matplotlib inline

In [2]:
df = pd.read_csv('df_2008_2018.csv')
df.head()

,gid,origin_id,uf,pathrow,mainclass,class_name,dsfnv,julday,view_date,ano,areakm_squared,scene_id,publish_ye,lon,lat,day,month,year,states
0,282618,580137.0,MA,22164,DESMATAMENTO,d2017,2016.0,221.0,2017-08-09 00:00:00,2017.0,0.012362,100624.0,2017-01-01 00:00:00,-46.147619,-5.115730,9,8,2017,Maranhao
1,593800,1114637.0,MT,22669,DESMATAMENTO,d2008,NaN,168.0,2008-06-16 00:00:00,2008.0,0.306214,527.0,2008-01-01 00:00:00,-54.881533,-12.708523,16,6,2008,Mato Grosso
2,816637,25644.0,PA,22562,DESMATAMENTO,d2018,NaN,236.0,2018-08-24 12:00:00,2018.0,0.067416,101109.0,2018-01-01 00:00:00,-50.980997,-3.530574,24,8,2018,Para
3,393766,724609.0,PA,22466,DESMATAMENTO,d2009,NaN,220.0,2009-08-08 00:00:00,2009.0,0.078053,879.0,2009-01-01 00:00:00,-50.702206,-8.960971,8,8,2009,Para
4,117567,366852.0,RR,23258,DESMATAMENTO,d2009,NaN,292.0,2009-10-19 00:00:00,2009.0,0.000377,974.0,2009-01-01 00:00:00,-60.262828,2.378392,19,10,2009,Roraima


In [3]:
df["target"] = df.lon.shift(-1)

In [4]:
df.dropna(inplace=True)

In [5]:
df.head()

,gid,origin_id,uf,pathrow,mainclass,class_name,dsfnv,julday,view_date,ano,areakm_squared,scene_id,publish_ye,lon,lat,day,month,year,states,target
0,282618,580137.0,MA,22164,DESMATAMENTO,d2017,2016.0,221.0,2017-08-09 00:00:00,2017.0,0.012362,100624.0,2017-01-01 00:00:00,-46.147619,-5.115730,9,8,2017,Maranhao,-54.881533
17,344217,664249.0,PA,22362,DESMATAMENTO,d2011,2010.0,206.0,2011-07-25 00:00:00,2011.0,0.147048,1332.0,2011-01-01 00:00:00,-49.056915,-3.308578,25,7,2011,Para,-51.776182
37,18301,100055.0,AM,461,DESMATAMENTO,d2017,2014.0,221.0,2017-08-09 00:00:00,2017.0,0.006747,100698.0,2017-01-01 00:00:00,-69.409826,-1.231334,9,8,2017,Amazonas,-54.474674
44,176445,463252.0,RO,23367,DESMATAMENTO,d2015,2014.0,252.0,2015-09-09 00:00:00,2015.0,0.096634,2548.0,2015-01-01 00:00:00,-64.903528,-10.731157,9,9,2015,Rondonia,-48.907652
48,673773,1266107.0,PA,22762,DESMATAMENTO,d2016,2014.0,197.0,2016-07-15 00:00:00,2016.0,0.089230,100233.0,2016-01-01 00:00:00,-53.879455,-2.343317,15,7,2016,Para,-61.644586


In [6]:
df.drop(["uf", "states", "mainclass", "class_name", "dsfnv", "view_date", "publish_ye", "gid", "origin_id", "scene_id", "pathrow", "ano"], axis=1, inplace=True)
#feature_names = data.columns
df.head()

,julday,areakm_squared,lon,lat,day,month,year,target
0,221.0,0.012362,-46.147619,-5.115730,9,8,2017,-54.881533
17,206.0,0.147048,-49.056915,-3.308578,25,7,2011,-51.776182
37,221.0,0.006747,-69.409826,-1.231334,9,8,2017,-54.474674
44,252.0,0.096634,-64.903528,-10.731157,9,9,2015,-48.907652
48,197.0,0.089230,-53.879455,-2.343317,15,7,2016,-61.644586


In [7]:
def train_test_split(data, perc):
    data = data.values
    n = int(len(data) * (1 - perc))
    return data[:n], data[n:]

In [8]:
train, test = train_test_split(df, 0.2)

In [9]:
print(len(df))
print(len(train))
print(len(test))

6202
4961
1241


In [10]:
X = train[:, :-1]
y = train[:, -1]

In [11]:
y

array([-54.88153302, -51.77618192, -54.474674  , ..., -51.68775393,
       -55.07351471, -50.59882613])

In [12]:
#df = df.drop(["uf", "states", "mainclass", "class_name", "dsfnv", "view_date", "publish_ye", "gid", "origin_id", "scene_id", "pathrow", "ano"])
#feature_names = data.columns
#df.head()
# removed inplace = True
# took it from here https://github.com/pandas-dev/pandas/issues/39431

KeyError: "['uf' 'states' 'mainclass' 'class_name' 'dsfnv' 'view_date' 'publish_ye'\n 'gid' 'origin_id' 'scene_id' 'pathrow' 'ano'] not found in axis"

In [13]:
from xgboost import XGBRegressor

model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
test[0]

array([ 1.12000000e+02,  3.27566792e-02, -4.99489169e+01, -3.76505912e+00,
        2.10000000e+01,  4.00000000e+00,  2.01600000e+03, -5.61309650e+01])

In [26]:
predictions = model.predict(val)  
(add .values)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6'] ['f0']
expected f4, f1, f3, f5, f6, f2 in input data

In [16]:
val = np.array(test[0, 0]).reshape(1, -1)

pred = model.predict(val)
print(pred)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6'] ['f0']
expected f4, f1, f3, f5, f6, f2 in input data

In [24]:
def xgb_predict(train, val):
    train = np.array(train)
    X, y = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
    model.fit(X, y)

    val = np.array(val).reshape(1, -1)
    pred = model.predict(val)
    return pred[0]

In [23]:
predictions = model.predict(test_data_process.values)  
(add .values)

NameError: name 'test_data_process' is not defined

In [19]:
xgb_predict(train, test[0, 0])

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6'] ['f0']
expected f4, f1, f3, f5, f6, f2 in input data

In [20]:
from sklearn.metrics import mean_squared_error


def validate(data, perc):
    predictions = []

    train, test = train_test_split(data, perc)

    history = [x for x in train]

    for i in range(len(test)):
        test_X, test_y = test[i, :-1], test[i, -1]

        pred = xgb_predict(history, test_X[0])
        predictions.append(pred)

        history.append(test[i])

    error = mean_squared_error(test[:, -1], predictions, squared=False)

    return error, test[:, -1], predictions

In [21]:
%%time
rmse, y, pred = validate(df, 0.2)

print(rmse)

C:\Users\besat\anaconda3\lib\site-packages\xgboost\data.py:104: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6'] ['f0']
expected f4, f1, f3, f5, f6, f2 in input data